In [3]:
from DAESIM_preprocess.silo_daily import silo_daily
import os
from DAESIM_preprocess.silo_daily import download_from_SILO
import xarray as xr

In [13]:
def silo_daily_singleyear(var="radiation", latitude=-34.3890427, longitude=148.469499, buffer=0.1, year="2020", silo_folder="."):
    """Select the region of interest from the Australia wide NetCDF file"""
    filename = os.path.join(silo_folder, f"{year}.{var}.nc")
    
    print(filename)
    if not os.path.exists(filename):
        download_from_SILO(var, year, silo_folder)

    # try:
    #     ds = xr.open_dataset(filename)
    # except Exception as e:
    #     # Likely no data for the specified year
    #     return None
    
    ds = xr.open_dataset(filename, cache=False)
    if 'crs' in list(ds.data_vars):
        ds = ds.drop_vars('crs')
    
    bbox = [longitude - buffer, latitude - buffer, longitude + buffer, latitude + buffer]
    ds_region = ds.sel(lat=slice(bbox[1], bbox[3]), lon=slice(bbox[0], bbox[2]))

    min_buffer_size = 0.03
    if buffer < min_buffer_size:
        # Find a single point but keep the lat and lon dimensions for consistency
        ds_region = ds.sel(lat=[latitude], lon=[longitude], method='nearest')
    
    return ds_region

In [17]:
ds = silo_daily_singleyear(year='2022')

./2022.radiation.nc
Downloaded ./2022.radiation.nc


In [16]:
import h5py

with h5py.File('/Users/yasar/paddock-ts-local/2020.radiation.nc', 'r') as f:
    print(list(f.keys()))
    # Or explore datasets
    for key in f.keys():
        print(f[key])

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = '/Users/yasar/paddock-ts-local/2020.radiation.nc', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
from dask.utils import SerializableLock


Note: you may need to restart the kernel to use updated packages.
